# Data Science  - Unit 2.1.2
Name: Michael Luo

Date: 2022/10/27

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'

# If you're working locally:
else:
    DATA_PATH = '../data/'

# Module Project: Regression II

In this project, you'll continue working with the New York City rent dataset you used in the last module project.

## Directions

The tasks for this project are as follows:

- **Task 1:** Import `csv` file using `wrangle` function.
- **Task 2:** Conduct exploratory data analysis (EDA), and modify `wrangle` function to engineer two new features.
- **Task 3:** Split data into feature matrix `X` and target vector `y`.
- **Task 4:** Split feature matrix `X` and target vector `y` into training and test sets.
- **Task 5:** Establish the baseline mean absolute error for your dataset.
- **Task 6:** Build and train a `Linearregression` model.
- **Task 7:** Calculate the training and test mean absolute error for your model.
- **Task 8:** Calculate the training and test $R^2$ score for your model.
- **Stretch Goal:** Determine the three most important features for your linear regression model.

**Note**

You should limit yourself to the following libraries for this project:

- `matplotlib`
- `numpy`
- `pandas`
- `sklearn`

In [ ]:
import numpy as np
import pandas as pd

#modeling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

#metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

#visualization
import matplotlib.pyplot as plt

# I. Wrangle Data

In [ ]:
def wrangle(filepath):
    df = pd.read_csv(filepath, parse_dates=['created'], infer_datetime_format=True, index_col='created')
    
    # Remove the most extreme 1% prices,
    # the most extreme .1% latitudes, &
    # the most extreme .1% longitudes
    df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
            (df['price'] <= np.percentile(df['price'], 99.5)) & 
            (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
            (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
            (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
            (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

    #drop rows with any NaN
    df.dropna(inplace=True)
    
    #DROP BAD DATA: columns with 0 bathroms and 0 bedrooms. 
    df = df[(df['bathrooms'] > 0) & (df['bedrooms'] > 0)]
    
    #drop columns with high cardinality or no clear value
    df = df.drop(columns=['description', 'display_address', 'street_address'])
    df['interest_level'] = df['interest_level'].map({'high': 2, 'medium':1, 'low':0})

    #FEATURE ENGINEERING #1: total perks
    perks = df.nunique()[df.nunique()<=2].index.tolist()
    perks.remove('pre-war') #not a perk
    df['perks'] = df[perks].sum(axis=1)   


    #FEATURE ENGINEERING #2: total rooms; bed + bath
    df['tot_room'] = df['bathrooms'] + df['bedrooms']

    return df

filepath = DATA_PATH + 'apartments/renthop-nyc.csv'

**Task 1:** Add the following functionality to the above `wrangle` function.

- The `'created'` column will parsed as a `DateTime` object and set as the `index` of the DataFrame. 
- Rows with `NaN` values will be dropped.

Then use your modified function to import the `renthop-nyc.csv` file into a DataFrame named `df`.

In [ ]:
df = wrangle(filepath)

**Task 2:** Using your `pandas` and dataviz skills decide on two features that you want to engineer for your dataset. Next, modify your `wrangle` function to add those features. 

**Note:** You can learn more about feature engineering [here](https://en.wikipedia.org/wiki/Feature_engineering). Here are some ideas for new features:

- Does the apartment have a description?
- Length of description.
- Total number of perks that apartment has.
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths).

In [ ]:
print(df.columns)

Index(['bathrooms', 'bedrooms', 'latitude', 'longitude', 'price',
       'interest_level', 'elevator', 'cats_allowed', 'hardwood_floors',
       'dogs_allowed', 'doorman', 'dishwasher', 'no_fee',
       'laundry_in_building', 'fitness_center', 'pre-war', 'laundry_in_unit',
       'roof_deck', 'outdoor_space', 'dining_room', 'high_speed_internet',
       'balcony', 'swimming_pool', 'new_construction', 'terrace', 'exclusive',
       'loft', 'garden_patio', 'wheelchair_access', 'common_outdoor_space',
       'perks', 'tot_room'],
      dtype='object')


In [ ]:
# Conduct your exploratory data analysis here, 
# and then modify the function above.
# df.head()
# display(df.info())
# display(df.nunique())


In [ ]:
df.head()

,bathrooms,bedrooms,latitude,longitude,price,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,...,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,perks,tot_room
created,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,40.7145,-73.9425,3000,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.5
2016-06-12 12:19:27,1.0,2,40.7947,-73.9667,5465,0,1,1,0,1,...,0,0,0,0,0,0,0,0,5,3.0
2016-04-17 03:26:41,1.0,1,40.7388,-74.0018,2850,2,0,0,1,0,...,0,0,0,0,0,0,0,0,3,2.0
2016-04-18 02:22:02,1.0,1,40.7539,-73.9677,3275,0,0,0,1,0,...,0,0,0,0,0,0,0,0,2,2.0
2016-04-28 01:32:41,1.0,4,40.8241,-73.9493,3350,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.0


# II. Split Data

**Task 3:** Split your DataFrame `df` into a feature matrix `X` and the target vector `y`. You want to predict `'price'`.

**Note:** In contrast to the last module project, this time you should include _all_ the numerical features in your dataset.

In [ ]:
target = 'price'
y = df[target]
X = df.drop(columns=[target])

**Task 4:** Split `X` and `y` into a training set (`X_train`, `y_train`) and a test set (`X_test`, `y_test`).

- Your training set should include data from April and May 2016. 
- Your test set should include data from June 2016.

In [ ]:
#Check years
# X.index.year.value_counts()
  #only 2016 data

#Check months
# X.index.month.value_counts()

In [ ]:
train_mask_x

array([False, False,  True, ...,  True,  True,  True])

In [ ]:
cutoff = '2016-06'
train_mask_x = X.index < cutoff
train_mask_y = y.index < cutoff

X_train, y_train = X[train_mask_x], y[train_mask_y]
X_test, y_test = X[~train_mask_x], y[~train_mask_y]

# III. Establish Baseline

**Task 5:** Since this is a **regression** problem, you need to calculate the baseline mean absolute error for your model. First, calculate the mean of `y_train`. Next, create a list `y_pred` that has the same length as `y_train` and where every item in the list is the mean. Finally, use `mean_absolute_error` to calculate your baseline.

In [ ]:
y_bar = y.mean()
y_pred_baseline = [y_bar] * len(y)
baseline_mae = mean_absolute_error(y, y_pred_baseline)
print('Baseline MAE:', baseline_mae)

Baseline MAE: 1263.5038027602855


# IV. Build Model

**Task 6:** Build and train a `LinearRegression` model named `model` using your feature matrix `X_train` and your target vector `y_train`.

In [ ]:
# Step 1: Import predictor class
#done

# Step 2: Instantiate predictor
model = LinearRegression()

# Step 3: Fit predictor on the (training) data
model.fit(X_train, y_train)

LinearRegression()

# V. Check Metrics

**Task 7:** Calculate the training and test mean absolute error for your model.

In [ ]:
dir(model)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_decision_function',
 '_estimator_type',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_preprocess_data',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_residues',
 '_set_intercept',
 '_validate_data',
 'coef_',
 'copy_X',
 'feature_names_in_',
 'fit',
 'fit_intercept',
 'get_params',
 'intercept_',
 'n_features_in_',
 'n_jobs',
 'normalize',
 'positive',
 'predict',
 'rank_',
 'score',
 'set_params',
 'singular_']

In [ ]:
#make dictionary for feature names and coefs
coefs = {}
for n, c in zip(model.feature_names_in_, model.coef_):
  coefs[n] = c

coefs

{'bathrooms': -841828491160127.8,
 'bedrooms': -841828491161463.1,
 'latitude': 1310.6970113860152,
 'longitude': -13759.425988814464,
 'interest_level': -430.24448960599733,
 'elevator': 89.4003472725947,
 'cats_allowed': -73.15342465352406,
 'hardwood_floors': -183.18826166916986,
 'dogs_allowed': 50.56227293571302,
 'doorman': 452.37909431275347,
 'dishwasher': 11.44820990671943,
 'no_fee': -187.95809951879767,
 'laundry_in_building': -146.72121200405047,
 'fitness_center': 21.38297384459778,
 'pre-war': -71.26439252779251,
 'laundry_in_unit': 463.2972010506661,
 'roof_deck': -230.44764073514605,
 'outdoor_space': -138.662109375,
 'dining_room': 211.68359375,
 'high_speed_internet': -363.75390625,
 'balcony': -132.005859375,
 'swimming_pool': 49.830078125,
 'new_construction': -183.478515625,
 'terrace': 141.3759765625,
 'exclusive': 33.384765625,
 'loft': 274.50439453125,
 'garden_patio': -115.947265625,
 'wheelchair_access': 156.5361328125,
 'common_outdoor_space': -160.7158203125

In [ ]:
training_mae = mean_absolute_error(y_train, model.predict(X_train))
test_mae = mean_absolute_error(y_test, model.predict(X_test))

print('Training MAE:', training_mae)
print('Test MAE:', test_mae)

Training MAE: 733.3456152511246
Test MAE: 733.0104360669859


**Task 8:** Calculate the training and test $R^2$ score for your model.

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
training_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print('Training MAE:', training_r2)
print('Test MAE:', test_r2)

Training MAE: 0.627866136531926
Test MAE: 0.6386426927036721


In [ ]:
#should be same as above
print(f'Training MAE: {model.score(X_train, y_train)}')
print(f'Testing MAE: {model.score(X_test, y_test)}')

Training MAE: 0.627866136531926
Testing MAE: 0.6386426927036721


# VI. Communicate Results

**Stretch Goal:** What are the three most influential coefficients in your linear model? You should consider the _absolute value_ of each coefficient, so that it doesn't matter if it's positive or negative.

In [ ]:
#Make dataframe of coef names and values

df_lf_coefs = pd.DataFrame({'name': model.feature_names_in_, 'coefficient': model.coef_})
df_lf_coefs['absolute_coef'] = df_lf_coefs['coefficient'].abs()

In [ ]:
#sort by highest absolute coefficient
top10_coefs = df_lf_coefs.sort_values(by='absolute_coef', ascending=False).head(n=10)['name'].to_list()

In [ ]:
#Highest absolute coefficients of the model come from total rooms, bedrooms, and bathrooms
#These are highly collinear?
top10_coefs

['tot_room',
 'bedrooms',
 'bathrooms',
 'longitude',
 'latitude',
 'laundry_in_unit',
 'doorman',
 'interest_level',
 'high_speed_internet',
 'loft']

In [ ]:
#play around with ols model
from statsmodels.formula.api import ols

model_ols = ols('price ~ bedrooms + bathrooms', data=df).fit()
print(model_ols.summary()) #r^2 = 0.499

model_ols = ols('price ~ tot_room', data=df).fit()
print(model_ols.summary()) #r^2 = 0.394

model_ols = ols('price ~ bathrooms', data=df).fit()
print(model_ols.summary()) #r^2 = 0.481

model_ols = ols('price ~ bedrooms', data=df).fit()
print(model_ols.summary()) #r^2 = 0.231

model_ols = ols('price ~ tot_room + bathrooms + bedrooms', data=df).fit()
print(model_ols.summary()) #r^2 = 0.499
#tot_room does not add any predictive value to the model. the variation is already described by bathrooms and bedrooms
#tot_room alone is a better predictor than bedrooms but worse than bathrooms


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.500
Model:                            OLS   Adj. R-squared:                  0.500
Method:                 Least Squares   F-statistic:                 1.908e+04
Date:                Thu, 27 Oct 2022   Prob (F-statistic):               0.00
Time:                        11:33:55   Log-Likelihood:            -3.2723e+05
No. Observations:               38104   AIC:                         6.545e+05
Df Residuals:                   38101   BIC:                         6.545e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    416.4800     18.938     21.991      0.0

In [ ]:
df.head()

,bathrooms,bedrooms,latitude,longitude,price,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,...,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,perks,tot_room
created,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,40.7145,-73.9425,3000,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.5
2016-06-12 12:19:27,1.0,2,40.7947,-73.9667,5465,0,1,1,0,1,...,0,0,0,0,0,0,0,0,5,3.0
2016-04-17 03:26:41,1.0,1,40.7388,-74.0018,2850,2,0,0,1,0,...,0,0,0,0,0,0,0,0,3,2.0
2016-04-18 02:22:02,1.0,1,40.7539,-73.9677,3275,0,0,0,1,0,...,0,0,0,0,0,0,0,0,2,2.0
2016-04-28 01:32:41,1.0,4,40.8241,-73.9493,3350,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.0
